In [ ]:
#imports required for flask part
from flask import Flask, render_template,request
import os
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
#download the summarizer
from summarizer import Summarizer
#import for SBERT summarizer
from summarizer.sbert import SBertSummarizer
#create an instance of the SBERT
model_bert = SBertSummarizer('paraphrase-MiniLM-L6-v2')

app = Flask(__name__)

def get_wiki_content(url):
    req_obj = requests.get(url)
    text = req_obj.text
    soup = BeautifulSoup(text)
    all_paras = soup.find_all("p")
    wiki_text = ''
    for para in all_paras:
        wiki_text += para.text 
    return wiki_text


@app.route("/")
def msg():
    return render_template('index.html')

@app.route("/summarize",methods=['POST','GET'])
def getSummary():
    body=request.form.get('data')
    if len(body)==0:
        if request.method == 'POST':
            f = request.files['file']
            if(f.filename==""):
                temp1=request.form['size']
                temp1=int(temp1)
                url = request.form.get("url")
                url_content = get_wiki_content(url)
                result = model_bert(url_content,num_sentences=temp1)
                return render_template('summary_bert.html',result=result)
            else:
                temp1=request.form['size']
                temp1=int(temp1)
                f.save(f.filename)
                f.seek(0)
                content = f.read()
                content = str(content, 'utf-8')
                result=model_bert(content,num_sentences=temp1)
                return render_template('summary_bert.html', result=result)
    else:
        temp1=request.form['size']
        temp1=int(temp1)
        body=request.form.get('data')
        result = model_bert(body, num_sentences=temp1)
        return render_template('summary_bert.html',result=result)

if __name__ =="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2023 01:08:55] "POST /summarize HTTP/1.1" 200 -
